In [1]:
!pip install requests beautifulsoup4 pandas nltk


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd '/content/drive/My Drive/Black-Coffer/TitleText'

[Errno 2] No such file or directory: '/content/drive/My Drive/Black-Coffer/TitleText'
/content


In [4]:
#import necessary pacakages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Read Excel file
df = pd.read_excel("/content/drive/My Drive/Black-Coffer/Input.xlsx")

# Define the folder path
folder_path = "/content/drive/My Drive/Black-Coffer/TitleText"

# Ensure the folder exists
os.makedirs(folder_path, exist_ok=True)

# Loop through each row
for index, row in df.iterrows():
    url = row['URL']
    url_id = row['URL_ID']

    # Make a request to the URL
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Check if request was successful
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url_id}: {e}")
        continue

    # Create a BeautifulSoup object
    try:
        soup = BeautifulSoup(response.content, 'html.parser')
    except Exception as e:
        print(f"Can't parse page of {url_id}: {e}")
        continue

    # Find title
    try:
        title = soup.find('h1').get_text().strip()
    except:
        print(f"Can't get title of {url_id}")
        title = "No Title"

    # Find text
    article = ""
    try:
        paragraphs = soup.find_all('p')
        if not paragraphs:
            print(f"No paragraphs found for {url_id}")
        for p in paragraphs:
            article += p.get_text() + "\n"
    except:
        print(f"Can't get text of {url_id}")
        article = "No Content"

    # Define file path
    file_name = os.path.join(folder_path, f"TitleText_{url_id}.txt")

    # Write title and text to file
    try:
        with open(file_name, 'w', encoding='utf-8') as file:
            file.write(title + '\n' + article)
        print(f"Saved: {file_name}")
    except Exception as e:
        print(f"Error writing file for {url_id}: {e}")


Saved: /content/drive/My Drive/Black-Coffer/TitleText/TitleText_Netclan20241017.txt
Saved: /content/drive/My Drive/Black-Coffer/TitleText/TitleText_Netclan20241018.txt
Saved: /content/drive/My Drive/Black-Coffer/TitleText/TitleText_Netclan20241019.txt
Saved: /content/drive/My Drive/Black-Coffer/TitleText/TitleText_Netclan20241020.txt
Saved: /content/drive/My Drive/Black-Coffer/TitleText/TitleText_Netclan20241021.txt
Saved: /content/drive/My Drive/Black-Coffer/TitleText/TitleText_Netclan20241022.txt
Saved: /content/drive/My Drive/Black-Coffer/TitleText/TitleText_Netclan20241023.txt
Saved: /content/drive/My Drive/Black-Coffer/TitleText/TitleText_Netclan20241024.txt
Saved: /content/drive/My Drive/Black-Coffer/TitleText/TitleText_Netclan20241025.txt
Saved: /content/drive/My Drive/Black-Coffer/TitleText/TitleText_Netclan20241026.txt
Saved: /content/drive/My Drive/Black-Coffer/TitleText/TitleText_Netclan20241027.txt
Saved: /content/drive/My Drive/Black-Coffer/TitleText/TitleText_Netclan20241

In [9]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:

# Directories
text_dir = "/content/drive/My Drive/Black-Coffer/TitleText"
stopwords_dir = "/content/drive/My Drive/Black-Coffer/StopWords"
sentment_dir = "/content/drive/My Drive/Black-Coffer/MasterDictionary"

# load all stop words from the stopwords directory and store in the set variable
stop_words = set()
for files in os.listdir(stopwords_dir):
  with open(os.path.join(stopwords_dir,files),'r',encoding='ISO-8859-1') as f:
    stop_words.update(set(f.read().splitlines()))

# load all text files  from the  directory and store in a list(docs)
docs = []
for text_file in os.listdir(text_dir):
  with open(os.path.join(text_dir,text_file),'r') as f:
    text = f.read()
#tokenize the given text file
    words = word_tokenize(text)
# remove the stop words from the tokens
    filtered_text = [word for word in words if word.lower() not in stop_words]
# add each filtered tokens of each file into a list
    docs.append(filtered_text)



# store positive, Negative words from the directory
pos=set()
neg=set()

for files in os.listdir(sentment_dir):
  if files =='positive-words.txt':
    with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1') as f:
      pos.update(f.read().splitlines())
  else:
    with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1') as f:
      neg.update(f.read().splitlines())

# now collect the positive  and negative words from each file
# calculate the scores from the positive and negative words
positive_words = []
Negative_words =[]
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []

#iterate through the list of docs
for i in range(len(docs)):
  positive_words.append([word for word in docs[i] if word.lower() in pos])
  Negative_words.append([word for word in docs[i] if word.lower() in neg])
  positive_score.append(len(positive_words[i]))
  negative_score.append(len(Negative_words[i]))
  polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
  subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))


In [11]:


# Average Sentence Length = the number of words / the number of sentences
# Percentage of Complex words = the number of complex words / the number of words
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

avg_sentence_length = []
Percentage_of_Complex_words  =  []
Fog_Index = []
complex_word_count =  []
avg_syllable_word_count =[]

stopwords = set(stopwords.words('english'))
def measure(file):
  with open(os.path.join(text_dir, file),'r') as f:
    text = f.read()
# remove punctuations
    text = re.sub(r'[^\w\s.]','',text)
# split the given text file into sentences
    sentences = text.split('.')
# total number of sentences in a file
    num_sentences = len(sentences)
# total words in the file
    words = [word  for word in text.split() if word.lower() not in stopwords ]
    num_words = len(words)

# complex words having syllable count is greater than 2
# Complex words are words in the text that contain more than two syllables.
    complex_words = []
    for word in words:
      vowels = 'aeiou'
      syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
      if syllable_count_word > 2:
        complex_words.append(word)

# Syllable Count Per Word
# We count the number of Syllables in each word of the text by counting the vowels present in each word.
#  We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.
    syllable_count = 0
    syllable_words =[]
    for word in words:
      if word.endswith('es'):
        word = word[:-2]
      elif word.endswith('ed'):
        word = word[:-2]
      vowels = 'aeiou'
      syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
      if syllable_count_word >= 1:
        syllable_words.append(word)
        syllable_count += syllable_count_word


    avg_sentence_len = num_words / num_sentences
    avg_syllable_word_count = syllable_count / len(syllable_words)
    Percent_Complex_words  =  len(complex_words) / num_words
    Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

    return avg_sentence_len, Percent_Complex_words, Fog_Index, len(complex_words),avg_syllable_word_count

# iterate through each file or doc
for file in os.listdir(text_dir):
  x,y,z,a,b = measure(file)
  avg_sentence_length.append(x)
  Percentage_of_Complex_words.append(y)
  Fog_Index.append(z)
  complex_word_count.append(a)
  avg_syllable_word_count.append(b)

In [12]:
# Word Count and Average Word Length Sum of the total number of characters in each word/Total number of words
# We count the total cleaned words present in the text by
# removing the stop words (using stopwords class of nltk package).
# removing any punctuations like ? ! , . from the word before counting.

def cleaned_words(file):
  with open(os.path.join(text_dir,file), 'r') as f:
    text = f.read()
    text = re.sub(r'[^\w\s]', '' , text)
    words = [word  for word in text.split() if word.lower() not in stopwords]
    length = sum(len(word) for word in words)
    average_word_length = length / len(words)
  return len(words),average_word_length

word_count = []
average_word_length = []
for file in os.listdir(text_dir):
  x, y = cleaned_words(file)
  word_count.append(x)
  average_word_length.append(y)


# To calculate Personal Pronouns mentioned in the text, we use regex to find
# the counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken
#  so that the country name US is not included in the list.
def count_personal_pronouns(file):
  with open(os.path.join(text_dir,file), 'r') as f:
    text = f.read()
    personal_pronouns = ["I", "we", "my", "ours", "us"]
    count = 0
    for pronoun in personal_pronouns:
      count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
  return count

pp_count = []
for file in os.listdir(text_dir):
  x = count_personal_pronouns(file)
  pp_count.append(x)

In [17]:
# Load Output Data Structure
output_df = pd.read_excel('/content/drive/My Drive/Black-Coffer/Output Data Structure.xlsx')

# Drop rows where URL_ID does not exist
output_df.set_index("URL_ID", inplace=True)  # Ensure URL_ID is the index
output_df.drop([44, 57, 144], errors='ignore', inplace=True)  # Ignore if not found
output_df.reset_index(inplace=True)  # Reset index

# Ensure variable lengths match
print("DataFrame rows:", len(output_df))
print("Variable lengths:", [len(var) for var in variables])

# Assign values safely
for i, var in enumerate(variables):
    if len(var) == len(output_df):  # Ensure equal lengths
        output_df.iloc[:, i+2] = var
    else:
        print(f"Skipping column {i+2} due to length mismatch")

# Save the dataframe to CSV
output_df.to_csv('/content/drive/My Drive/Black-Coffer/Output_Data.csv', index=False)
print("Saved Output_Data.csv successfully!")


DataFrame rows: 147
Variable lengths: [147, 147, 147, 147, 147, 147, 147, 147, 147, 147, 147, 147, 147]
Saved Output_Data.csv successfully!
